In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from networkx import (draw, DiGraph, Graph)

Import bowtie output file

In [55]:
alignedUMI = np.loadtxt("/camp/lab/znamenskiyp/home/shared/projects/turnerb_MAPseq/Sequencing/Processed_data/BRAC5676.1h/trial/unzipped/barcodesplitter/bowtiealignment1_4.txt", dtype=int)

print(alignedUMI)


[[     1      1]
 [     2      2]
 [     2      3]
 ...
 [274748 274748]
 [274750 274750]
 [274751 274751]]


Now group according to connections within each row

In [61]:
G=nx.Graph()
G.add_edges_from(alignedUMI)
print(sorted(nx.connected_components(G), key = len, reverse=True))


[{66221, 62149, 61625, 61626, 61627, 61628, 61629, 61630, 61631, 61632, 61633, 61634, 61635, 61636, 61746, 61747, 61748, 61749, 61750, 61751, 61752, 61753, 61757, 61758, 61759, 61760, 61761, 61762, 61765, 61766, 61767, 61768, 61769, 61778, 61779, 61780, 61781, 61782, 61783, 61788, 61790, 61791, 61792, 61793, 61794, 61795, 61796, 61797, 61798, 61799, 61803, 61805, 61806, 61808, 61814, 61815, 61820, 61821, 61822, 61823, 61824, 61825, 61826, 61827, 61828, 61829, 61830, 61831, 61832, 61834, 61835, 61836, 61837, 61838, 61839, 61846, 61847, 61848, 61849, 61850, 61851, 61852, 61853, 61854, 61855, 61857, 61858, 61859, 61860, 61861, 61862, 61867, 61868, 61869, 61870, 61871, 61872, 61873, 61874, 61875, 61876, 61877, 61878, 61879, 61880, 61881, 61882, 61883, 61884, 61885, 61886, 61887, 61888, 61889, 61890, 61891, 61892, 61893, 61894, 61895, 61896, 61897, 61898, 61899, 61900, 61901, 61902, 61903, 61904, 61905, 61906, 61907, 61908, 61909, 61910, 61912, 61913, 61914, 61915, 61917, 61918, 61919, 6192

Now take first UMI in each graph component and count number within